In [ ]:
#importing libraries
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from skimage import io 
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import torchvision.transforms as tf
import os
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#total images in dataset = 1540
#for each image 35 divisions made, and in total 44 images, so 35*44 = 1540 
#after excluding 1.0.0 , 2.0.0 , 3.0.0 , 4.0.0 we will have 140 images less
#so in total  = 1400 images
#80% of 1400 = 1120

class MedicalImage(Dataset):
  
  def __init__(self, data_path):
    self.path = data_path
    all_files = os.listdir(data_path)
    l = len(all_files)
    self.store10x = [] # stores 35*8 = 280, 10x images

    for item in all_files:

      if (item.split("_")[1]=='1' or item.split("_")[1]=='2') and item.split("_")[2]=='0': 
        self.store10x.append(item) # 280 (10x images) are filtered
      else:
          pass
          
    print(len(self.store10x))

  def __len__(self):
    x = len(self.store10x)
    return (x)

  def __getitem__(self,index):
    img_path = f'{self.path}{self.store10x[index]}'
    img = Image.open(img_path)
    z = self.store10x[index].split('_')[2]
    if (z==0):
      sf = 2.5
    else:
      sf = 10.0
    img = img.resize((256,256),resample=Image.BILINEAR)
    w,h = img.size

    blur_img = img.resize((int(w/sf),int(h/sf)),resample=Image.BILINEAR)
    blur_img = blur_img.resize((w,h),resample=Image.BILINEAR)
    
    input_img = tf.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])(tf.ToTensor()(blur_img))
    target_img = tf.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])(tf.ToTensor()(img))
    return(input_img, target_img)

data = MedicalImage('/content/drive/MyDrive/Medical Image Dataset/')  # PRETAM'S MACHINE
#data = MedicalImage('/content/drive/MyDrive/Cropped Medical Images/')  # SWARNENDU'S MACHINE
inp,tar = data[24]
print(inp.shape, tar.shape)

280
torch.Size([3, 256, 256]) torch.Size([3, 256, 256])


In [ ]:
# Hyperparameters
LEARNING_RATE = 2e-4 
BATCH_SIZE = 2
IMAGE_SIZE = 256
CHANNELS_IMG = 3
IN_CHANNNELS = 3
OUT_CHANNELS = 8
NUM_EPOCHS = 30

In [ ]:
#Load Data
from torch.utils.data import DataLoader

trainloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

testloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import torch.nn.functional as F
from torch.nn import Module

class HierConv(Module):    # HIERARCHICAL CONVOLUTIONS
    def __init__(self,IC,OC,level=5,remap_K=1,remap_S=1,remap_P=0):
        super(HierConv, self).__init__()
        self.level = level
        assert(self.level>=5 and self.level%2==1), "level should be a odd number greater than or equal to 5"
        self.layers = {}
        for l in range(5,self.level+1,2):
            self.layers[str(l)]=nn.ModuleList([nn.Conv2d(IC,OC,k,1,k//2).cuda() for k in range(3,l+1,2)])
        self.output_dim=0
        for i in range(5, self.level+1, 2):
            for j in range (1, i-1, 2):
                self.output_dim += j**2 
        self.remap = nn.Conv2d(OC*self.output_dim,OC,remap_K,remap_S,remap_P,1,OC)
    def forward(self, x):
        outs={}
        for l in range(5,self.level+1,2):
            outs[str(l)]=[self.layers[str(l)][i](x) for i in range(l//2)]
        B,C,H,W = outs['5'][0].shape
        final_out=torch.zeros(B,C*self.output_dim,H,W).cuda()
        for b in range(B):
            for c in range(C):
                stackedFeatures = []
                for l in range(5,self.level+1,2):
                    d=-1
                    stackedFeatures.append(outs[str(l)][d][b,c,:,:])
                    d-=1
                    for p in range(3,l,2):
                        for sh in range(0,p):                                                                                
                            for sw in range(0,p):
                                stackedFeatures.append(F.pad(outs[str(l)][d][b,c,:,:],(sh,p-1-sh,sw,p-1-sw))[p//2:-(p//2),p//2:-(p//2)]) 
                        d-=1
                m = self.output_dim 
                final_out[b,m*c:m*c+m,:,:]=torch.stack(stackedFeatures, 0)
        output = self.remap(final_out)
        return(output)

hc = HierConv(3,8,5).cuda()
input = torch.randn(4,3,256,256).cuda() #b,c,h,w
output = hc(input)
output.shape 

torch.Size([4, 8, 256, 256])

In [ ]:
class SelfAttentionGen(nn.Module):
  def __init__(self, in_dim):
    super(SelfAttentionGen, self).__init__()

    self.query_conv = nn.Conv2d(in_dim, in_dim//8, kernel_size=1)
    self.key_conv = nn.Conv2d(in_dim, in_dim//8, kernel_size=1)
    self.value_conv = nn.Conv2d(in_dim, in_dim, kernel_size=1)

    self.gamma = nn.Parameter(torch.zeros(1))
    
    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x):
    batch_size, channels, height, width = x.size()
    print(11)
    proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
    print(12)
    proj_key = self.key_conv(x).view(batch_size, -1, width * height)
    print(13)
    energy = torch.bmm(proj_query, proj_key)
    print(14)
    attention = self.softmax(energy)
    print(15)
    proj_value = self.value_conv(x).view(batch_size, -1, width * height)
    print(16)
    out = torch.bmm(proj_value, attention.permute(0, 2, 1))
    print(17)
    out = out.view(batch_size, channels, height, width)
    print(18)
    out = self.gamma * out + x
    print(19)
    return out

In [ ]:
class ConvBlock(nn.Module):
  def __init__(self,IC,OC,K,S,P):
    super(ConvBlock, self).__init__()
    self.layers = nn.Sequential(nn.Conv2d(IC,OC,K,S,P),
                      nn.BatchNorm2d(OC),
                      nn.LeakyReLU(),
                      nn.Conv2d(OC,OC,K,S,P),
                      nn.BatchNorm2d(OC),
                      nn.LeakyReLU(),
                      #SelfAttentionGen(OC),
                      HierConv(OC,OC),
                      nn.BatchNorm2d(OC),
                      nn.LeakyReLU())
    
  def forward(self, x):
    return(self.layers(x))

class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.conv1 = ConvBlock(3,8,3,1,1)
    self.mp1 = nn.MaxPool2d(2,2)
    self.conv2 = ConvBlock(8,16,3,1,1)
    self.mp2 = nn.MaxPool2d(2,2)
    self.conv3 = ConvBlock(16,24,3,1,1)
    self.mp3 = nn.MaxPool2d(2,2)
    self.conv4 = ConvBlock(24,32,3,1,1)
    self.mp4 = nn.MaxPool2d(2,2)
    self.conv5 = ConvBlock(32,40,3,1,1)
    self.up1 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.conv6 = ConvBlock(40,32,3,1,1)
    self.up2 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.squeeze1 = ConvBlock(64,32,3,1,1)
    self.conv7 = ConvBlock(32,24,3,1,1)
    self.up3 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.squeeze2 = ConvBlock(48,24,3,1,1)
    self.conv8 = ConvBlock(24,16,3,1,1)
    self.up4 = nn.UpsamplingBilinear2d(scale_factor=2)
    self.squeeze3 = ConvBlock(32,16,3,1,1)
    self.conv9 = ConvBlock(16,8,3,1,1)
    self.out = nn.Conv2d(8,3,3,1,1)

  def forward(self,x):                                    # 3 * 256 * 256
    c1=self.conv1(x)                                      # 8 * 256 * 256
    cm1=self.mp1(c1)                                      # 8 * 128 * 128
    c2=self.conv2(cm1)                                    # 16 * 128 * 128
    cm2 =self.mp2(c2)                                     # 16 * 64 * 64
    c3=self.conv3(cm2)                                    # 24 * 64 * 64
    cm3 = self.mp3(c3)                                    # 24 * 32 * 32
    c4=self.conv4(cm3)                                    # 32 * 32 * 32
    cm4 = self.mp4(c4)                                    # 32 * 16 * 16
    mid = self.conv5(cm4)                                 # 40 * 16 * 16
    u1 = self.up1(mid)                                    # 40 * 32 * 32
    uc1 = torch.cat((self.conv6(u1),c4), dim=1)           # 40 * 32 * 32
    uc1s = self.squeeze1(uc1)                             # 32 * 32 * 32
    u2 = self.up2(uc1s)                                   # 32 * 64 * 64
    uc2 = torch.cat((self.conv7(u2), c3), dim=1)          # 32 * 64 * 64
    uc2s = self.squeeze2(uc2)                             # 24 * 64 * 64
    u3 = self.up3(uc2s)                                   # 24 * 128 * 128
    uc3 = torch.cat((self.conv8(u3), c2), dim=1)          # 24 * 128 * 128
    uc3s = self.squeeze3(uc3)                             # 16 * 128 * 128
    u4 = self.up4(uc3s)                                   # 16 * 256 * 256
    uc4 = self.conv9(u4)                                  # 8 * 256 * 256
    ucfinal = self.out(uc4)                               # 3 * 256 * 256
    return(torch.tanh(ucfinal))
    #norm = ucfinal.pow(2).sum(dim=1, keepdim=True).sqrt()
    #out = (ucfinal/norm)

In [ ]:
class SelfAttentionDisc(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttentionDisc, self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        #print(1)
        batch_size, channels, height, width = x.size()
        #print(2)
        proj_query = self.query_conv(x).view(batch_size, -1, height * width).permute(0, 2, 1)
        #print(3)
        proj_key = self.key_conv(x).view(batch_size, -1, height * width)
        #print(4)
        energy = torch.bmm(proj_query, proj_key)
        #print(5)
        attention = F.softmax(energy, dim=-1)
        #print(6)
        proj_value = self.value_conv(x).view(batch_size, -1, height * width)
        #print(7)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        #print(8)
        out = out.view(batch_size, channels, height, width)
        #print(9)
        out = self.gamma * out + x
        #print(10)
        return out

In [ ]:
class DiscConvBlock(nn.Module):
  def __init__(self,IC,OC,K,S,P):
    super(DiscConvBlock , self).__init__()
    self.layers = nn.Sequential(nn.Conv2d(IC,OC,K,S,P),
                                nn.BatchNorm2d(OC),
                                nn.ReLU())
    
  def forward(self, x):
    return(self.layers(x))

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator,self).__init__()
    self.conv1=DiscConvBlock(3,8,3,1,1)
    #self.attention1 = SelfAttentionDisc(8)
    self.mp1 = nn.MaxPool2d(2,2)
    self.conv2=DiscConvBlock(8,16,3,1,1)
    #self.attention2 = SelfAttentionDisc(16)
    self.mp2 = nn.MaxPool2d(2,2)
    self.conv3=DiscConvBlock(16,32,3,1,1)
    #self.attention3 = SelfAttentionDisc(32)
    self.mp3 = nn.MaxPool2d(2,2)
    self.conv4=DiscConvBlock(32,64,3,1,1)
    #self.attention4 = SelfAttentionDisc(64)
    self.mp4 = nn.MaxPool2d(2,2)
    self.conv5=DiscConvBlock(64,128,3,1,1)
    self.attention5 = SelfAttentionDisc(128)

    self.avgpl = nn.AvgPool2d(16,16)
    self.linear1 = nn.Linear(128,64)
    self.linear2 = nn.Linear(64,1)

  def forward(self,x):
    c1 = self.conv1(x)       # 3 * 256 * 256
    #c1 = self.attention1(c1) # apply self-attention
    cm1 = self.mp1(c1)       # 3 * 128 * 128
    c2 = self.conv2(cm1)     # 8 * 128 * 128
    #c2 = self.attention2(c2) # apply self-attention
    cm2 = self.mp2(c2)       # 8 * 64 * 64
    c3 = self.conv3(cm2)     # 16 * 64 * 64
    #c3 = self.attention3(c3) # apply self-attention
    cm3 = self.mp3(c3)       # 16 * 32 * 32
    c4 = self.conv4(cm3)     # 32 * 32 * 32
    #c4 = self.attention4(c4)
    cm4 = self.mp4(c4)       # 32 * 16 * 16
    c5 = self.conv5(cm4)     # 64 * 16 * 16
    #c5 = self.attention5(c5)
    avg = self.avgpl(c5).squeeze() # 128 * 1 * 1
    l1 = self.linear1(avg)   # 64 * 1 * 1
    l2 = self.linear2(l1)    # 1
    return(l2)

#isc = Discriminator().to(device)
#inp = torch.randn(4,3,256,256).to(device)
#out = disc(inp)
#print(out.shape)
#save_image(inp, "inp.png")
#save_image(out, "out.png")

In [ ]:
gen = Generator().to(device)
disc = Discriminator().to(device)

optGen = optim.Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999))
optDisc = optim.Adam(disc.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
# Original Training Loop without WGAN
lossWeight = 0.990

for epoch in range(NUM_EPOCHS):
  gen.train()
  disc.train()
  totalLoss = 0
  epochLoss = 0
  perEpochLoss = 0
  l = len(trainloader)
  for i, (bI,sI) in enumerate(trainloader):
    if torch.cuda.is_available():
      bI, sI = bI.to(device), sI.to(device)

      # Training Discriminator  
      optDisc.zero_grad()  
      discOutReal = disc(sI)
      genOutD = gen(bI)  
      discOutFake = disc(genOutD)
      lossReal = critDisc(discOutReal, torch.ones_like(discOutReal).to(device))
      lossFake = critDisc(discOutFake, torch.zeros_like(discOutFake).to(device))
      lossDisc = (lossReal+lossFake)/2.0
      lossDisc.backward(retain_graph=True)
      optDisc.step()

      # Training Generator
      optGen.zero_grad()
      genOut = gen(bI)    
      discOut = disc(genOut)
      lossDiscGen = critDisc(discOut, torch.ones_like(discOut).to(device))
      lossGen = (1-lossWeight)*lossDiscGen + lossWeight*critGen(genOut, sI)
 
      lossGen.backward()
      optGen.step()

      totalLoss += (lossDisc.detach() + lossGen.detach())

      torch.cuda.empty_cache()
      if(i==l-2):

        save_image(bI, "/content/drive/MyDrive/finalTraining/40xImageTrain/%03d_blurimage.png"%epoch, nrow=8)
        save_image(sI, "/content/drive/MyDrive/finalTraining/40xImageTrain/%03d_sharpimage.png"%epoch, nrow=8)
        save_image(genOutD, "/content/drive/MyDrive/finalTraining/40xImageTrain/%03d_genimage.png"%epoch, nrow=8)   

  sampleLoss = totalLoss/(len(data))
  print(f"Epoch[{epoch+1}/{NUM_EPOCHS}] \
                Loss D: {lossDisc:.4f}, \
                loss G: {lossGen:.4f}, \
                loss per sample: {sampleLoss:.4f}\n")

In [ ]:
# 1st attempt
# New Modifications to training loop with Gradient Penalty(WGAN-GP)

gen = Generator().to(device)
disc = Discriminator().to(device)

# Hyperparameters
n_critic = 5
lambda_gp = 10.0
l = len(trainloader)

# Defining the optimizers for the generator and discriminator
gen_optimizer = optim.Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999))
disc_optimizer = optim.Adam(disc.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Defining the WGAN-GP loss function
def wgan_gp_loss(scores_real, scores_fake, real_images, fake_images, disc):
    batch_size = real_images.size(0)
    epsilon = torch.rand(batch_size, 1, 1, 1).to(device)
    interpolated_images = (epsilon * real_images + (1 - epsilon) * fake_images).requires_grad_(True)
    interpolated_scores = disc(interpolated_images)
    grad_outputs = torch.ones_like(interpolated_scores).to(device)
    gradients = torch.autograd.grad(outputs=interpolated_scores, inputs=interpolated_images, grad_outputs=grad_outputs,
                                    create_graph=True, retain_graph=True)[0]
    gradient_penalty = ((gradients.view(batch_size, -1).norm(2, dim=1) - 1) ** 2).mean()
    wgan_loss = torch.mean(scores_fake) - torch.mean(scores_real)
    wgan_gp_loss = wgan_loss + lambda_gp * gradient_penalty
    return wgan_gp_loss

# Training the GAN model with WGAN-GP
for epoch in range(NUM_EPOCHS):
    for i, (blur_images, sharp_images) in enumerate(trainloader):

        # Training the discriminator
        disc_optimizer.zero_grad()
        real_scores = disc(sharp_images.to(device))
        fake_images = gen(blur_images.to(device))
        fake_scores = disc(fake_images.detach())
        disc_loss = wgan_gp_loss(real_scores, fake_scores, sharp_images.to(device), fake_images.detach(), disc)
        disc_loss.backward()
        disc_optimizer.step()

        # Training the generator
        if i % n_critic == 0:
            gen_optimizer.zero_grad()
            fake_scores = disc(fake_images)
            gen_loss = -torch.mean(fake_scores)
            gen_loss.backward()
            gen_optimizer.step()

        if(i==l-2):
                save_image(blur_images, "/content/drive/MyDrive/finalTraining/3.10xGPmodified/%03d_blurimage.png"%epoch, nrow=8)
                save_image(sharp_images, "/content/drive/MyDrive/finalTraining/3.10xGPmodified/%03d_sharpimage.png"%epoch, nrow=8)
                save_image(fake_images, "/content/drive/MyDrive/finalTraining/3.10xGPmodified/%03d_genimage.png"%epoch, nrow=8)

    # Printing the epoch losses
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}: Gen Loss: {gen_loss.item():.4f}, Disc Loss: {disc_loss.item():.4f}")

Epoch 1/30: Gen Loss: -0.8049, Disc Loss: -0.7329
Epoch 2/30: Gen Loss: 0.1747, Disc Loss: -2.3318
Epoch 3/30: Gen Loss: 2.4879, Disc Loss: -7.9104
Epoch 4/30: Gen Loss: 6.5007, Disc Loss: -17.5248
Epoch 5/30: Gen Loss: 7.4579, Disc Loss: -35.0493
Epoch 6/30: Gen Loss: 10.4767, Disc Loss: -20.2416
Epoch 7/30: Gen Loss: 7.4447, Disc Loss: -28.5406
Epoch 8/30: Gen Loss: 2.2457, Disc Loss: -12.5903
Epoch 9/30: Gen Loss: -0.9436, Disc Loss: -17.4500
Epoch 10/30: Gen Loss: 2.3176, Disc Loss: -18.2446
Epoch 11/30: Gen Loss: 1.0535, Disc Loss: -6.6858
Epoch 12/30: Gen Loss: -5.4801, Disc Loss: -6.4486


In [ ]:
# 2nd attempt with WGAN
# Hyperparameters
lambda_gp = 10.0
num_critic_iter = 5

for epoch in range(NUM_EPOCHS):
    gen.train()
    disc.train()
    totalLoss = 0
    epochLoss = 0
    perEpochLoss = 0
    l = len(trainloader)

    for i, (bI,sI) in enumerate(trainloader):
        if torch.cuda.is_available():
            bI, sI = bI.to(device), sI.to(device)

            # Training Discriminator
            for _ in range(num_critic_iter):
                optDisc.zero_grad()
                with torch.no_grad():
                    genOutD = gen(bI)
                discOutReal = disc(sI)
                discOutFake = disc(genOutD)

                # Computing Wasserstein distance
                wass_dist = discOutReal.mean() - discOutFake.mean()

                # Computing gradient penalty
                alpha = torch.rand(sI.size(0), 1, 1, 1).to(device)
                interpolated = (alpha * sI + (1 - alpha) * genOutD).requires_grad_(True)
                discOutInterpolated = disc(interpolated)
                grad = torch.autograd.grad(outputs=discOutInterpolated, inputs=interpolated,
                                           grad_outputs=torch.ones_like(discOutInterpolated),
                                           create_graph=True, retain_graph=True)[0]
                grad = grad.view(grad.size(0), -1)
                grad_norm = grad.norm(2, dim=1)
                gradient_penalty = lambda_gp * ((grad_norm - 1) ** 2).mean()

                # Computing the total loss
                lossDisc = -wass_dist + gradient_penalty
                lossDisc.backward()
                optDisc.step()

            # Training Generator
            optGen.zero_grad()
            genOut = gen(bI)
            discOut = disc(genOut)
            lossGen = -discOut.mean()
            lossGen.backward()
            optGen.step()

            totalLoss += (lossDisc.detach() + lossGen.detach())

            torch.cuda.empty_cache()

            if(i==l-2):
                save_image(bI, "/content/drive/MyDrive/finalTraining/2.10xGP/%03d_blurimage.png"%epoch, nrow=8)
                save_image(sI, "/content/drive/MyDrive/finalTraining/2.10xGP/%03d_sharpimage.png"%epoch, nrow=8)
                save_image(genOutD, "/content/drive/MyDrive/finalTraining/2.10xGP/%03d_genimage.png"%epoch, nrow=8)

    sampleLoss = totalLoss/(len(data))
    print(f"Epoch[{epoch+1}/{NUM_EPOCHS}] \
                Loss D: {lossDisc:.4f}, \
                loss G: {lossGen:.4f}, \
                loss per sample: {sampleLoss:.4f}\n")

Epoch[1/30]                 Loss D: -37.2030,                 loss G: 22.9717,                 loss per sample: -0.9349

Epoch[2/30]                 Loss D: -30.1968,                 loss G: 22.0159,                 loss per sample: -4.3347

Epoch[3/30]                 Loss D: -51.2278,                 loss G: 18.2418,                 loss per sample: -2.0244

Epoch[4/30]                 Loss D: -37.0505,                 loss G: 22.3241,                 loss per sample: 1.6311

Epoch[5/30]                 Loss D: -3.1104,                 loss G: 5.1461,                 loss per sample: 50.8786

Epoch[6/30]                 Loss D: -2.2347,                 loss G: 6.8250,                 loss per sample: 2.3301

Epoch[7/30]                 Loss D: -4.6622,                 loss G: 6.2931,                 loss per sample: 2.7335

Epoch[8/30]                 Loss D: -10.5655,                 loss G: 8.6185,                 loss per sample: 0.2081

Epoch[9/30]                 Loss D: -11.947

In [ ]:
# 3rd attempt
#TRAINING LOOP

lossWeight = 0.990

for epoch in range(NUM_EPOCHS):
  gen.train()
  disc.train()
  totalLoss = 0
  epochLoss = 0
  perEpochLoss = 0
  l = len(trainloader)
  for i, (bI,sI) in enumerate(trainloader):
    if torch.cuda.is_available():
      bI, sI = bI.to(device), sI.to(device)

      # Training Discriminator  
      optDisc.zero_grad()  
      discOutReal = disc(sI)
      genOutD = gen(bI)  
      discOutFake = disc(genOutD)
      lossReal = discOutReal.mean()
      lossFake = discOutFake.mean()
      gradient_penalty = calc_gradient_penalty(disc, sI, genOutD)
      lossDisc = lossFake - lossReal + 10 * gradient_penalty
      lossDisc.backward(retain_graph=True)
      optDisc.step()

      # Training Generator
      optGen.zero_grad()
      genOut = gen(bI)    
      discOut = disc(genOut)
      lossDiscGen = discOut.mean()
      lossGen = (1-lossWeight)*lossDiscGen + lossWeight*F.mse_loss(genOut, sI)
 
      lossGen.backward()
      optGen.step()

      totalLoss += (lossDisc.detach() + lossGen.detach())

      torch.cuda.empty_cache()
      if(i==l-2):

        save_image(bI, "/content/drive/MyDrive/finalTraining/10xImageGradientPenalty/%03d_blurimage.png"%epoch, nrow=8)
        save_image(sI, "/content/drive/MyDrive/finalTraining/10xImageGradientPenalty/%03d_sharpimage.png"%epoch, nrow=8)
        save_image(genOutD, "/content/drive/MyDrive/finalTraining/10xImageGradientPenalty/%03d_genimage.png"%epoch, nrow=8)   

  sampleLoss = totalLoss/(len(data))
  print(f"Epoch[{epoch+1}/{NUM_EPOCHS}] \
                Loss D: {lossDisc:.4f}, \
                loss G: {lossGen:.4f}, \
                loss per sample: {sampleLoss:.4f}\n")

Epoch[1/30]                 Loss D: -0.3038,                 loss G: 0.2278,                 loss per sample: 3.5087

Epoch[2/30]                 Loss D: -35.8170,                 loss G: -0.1808,                 loss per sample: -8.4459

Epoch[3/30]                 Loss D: -97.5406,                 loss G: -0.5188,                 loss per sample: -31.1443

Epoch[4/30]                 Loss D: -192.5621,                 loss G: -0.9930,                 loss per sample: -68.1617

Epoch[5/30]                 Loss D: -314.9494,                 loss G: -1.6549,                 loss per sample: -118.1484

Epoch[6/30]                 Loss D: -460.7984,                 loss G: -2.4020,                 loss per sample: -185.1122

Epoch[7/30]                 Loss D: -617.3685,                 loss G: -3.1913,                 loss per sample: -263.6455

Epoch[8/30]                 Loss D: -839.8500,                 loss G: -4.1734,                 loss per sample: -353.6237

Epoch[9/30]         

KeyboardInterrupt: ignored

In [ ]:
# 4th attempt 
#TRAINING LOOP
clip_value = 0.01
n_critic = 5
lossWeight = 0.990

for epoch in range(NUM_EPOCHS):
  gen.train()
  disc.train()
  totalLoss = 0
  epochLoss = 0
  perEpochLoss = 0
  l = len(trainloader)
  for i, (bI,sI) in enumerate(trainloader):
    if torch.cuda.is_available():
      bI, sI = bI.to(device), sI.to(device)

      # Training Discriminator
      for n in range(n_critic): # discriminator will be trained n_critic times before updating Generator's weights
          optDisc.zero_grad()  
          discOutReal = disc(sI)
          genOutD = gen(bI)  
          discOutFake = disc(genOutD.detach())
          lossDisc = wasserstein_loss(discOutReal - discOutFake, torch.ones_like(discOutReal).to(device))
          lossDisc.backward()
          optDisc.step()

          # Clip weights
          for p in disc.parameters():
              p.data.clamp_(-clip_value, clip_value)

      # Training Generator
      optGen.zero_grad()
      genOut = gen(bI)    
      discOut = disc(genOut)
      lossGen = -wasserstein_loss(discOut, torch.ones_like(discOut).to(device))
      lossGen.backward()
      optGen.step()

      totalLoss += (lossDisc.detach() + lossGen.detach())

      torch.cuda.empty_cache()
      if(i==l-2):

        save_image(bI, "/content/drive/MyDrive/finalTraining/10xHierConv()Image/%03d_blurimage.png"%epoch, nrow=8)
        save_image(sI, "/content/drive/MyDrive/finalTraining/10xHierConv()Image/%03d_sharpimage.png"%epoch, nrow=8)
        save_image(genOutD, "/content/drive/MyDrive/finalTraining/10xHierConv()Image/%03d_genimage.png"%epoch, nrow=8)   

  sampleLoss = totalLoss/(len(data))
  print(f"Epoch[{epoch+1}/{NUM_EPOCHS}] \
                Loss D: {lossDisc:.4f}, \
                loss G: {lossGen:.4f}, \
                loss per sample: {sampleLoss:.4f}\n")

In [ ]:
from torchsummary import summary

summary(disc,(3,256,256))

1
2
3
4
5
6
7
8
9
10
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 256, 256]             224
       BatchNorm2d-2          [-1, 8, 256, 256]              16
              ReLU-3          [-1, 8, 256, 256]               0
     DiscConvBlock-4          [-1, 8, 256, 256]               0
         MaxPool2d-5          [-1, 8, 128, 128]               0
            Conv2d-6         [-1, 16, 128, 128]           1,168
       BatchNorm2d-7         [-1, 16, 128, 128]              32
              ReLU-8         [-1, 16, 128, 128]               0
     DiscConvBlock-9         [-1, 16, 128, 128]               0
        MaxPool2d-10           [-1, 16, 64, 64]               0
           Conv2d-11           [-1, 32, 64, 64]           4,640
      BatchNorm2d-12           [-1, 32, 64, 64]              64
             ReLU-13           [-1, 32, 64, 64]               0
    DiscConvBlock-

In [ ]:
summary(gen,(3,256,256))

OutOfMemoryError: ignored

In [ ]:
for params in disc.parameters():
  print(np.prod(np.array(params.shape)))

216
8
8
8
1
8
1
8
1
64
8
1152
16
16
16
1
32
2
32
2
256
16
4608
32
32
32
1
128
4
128
4
1024
32
18432
64
64
64
1
512
8
512
8
4096
64
73728
128
128
128
1
2048
16
2048
16
16384
128
8192
64
64
1


In [ ]:
genHierConv = Generator().to(device)
mo = torch.load('/content/drive/MyDrive/finalTraining/HierConv()Model.pt')
model = genHierConv.load_state_dict(mo)

In [ ]:
torch.save(gen.cpu().state_dict(),"/content/drive/MyDrive/finalTraining/HierConv()Model.pt")

In [ ]:
gen1 = Generator().to(device)
mo = torch.load('//content/drive/MyDrive/GATE 2021/10xSavedModel.pt')
model = gen1.load_state_dict(mo)

In [ ]:
del(disc)
torch.cuda.empty_cache()